## Toronto Neighbourhood data

#### First, intall and import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
from pandas.io.html import read_html

#### Now, read the Canada postal code Wiki web page 

In [3]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,  attrs={"class":"wikitable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


#### Import the Toronto data into pandas DataFrame

In [7]:
toronto_df = pd.DataFrame(wikitables[0])
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Let's check DataFrame's dimms

In [5]:
toronto_df.shape

(288, 3)

#### We have to keep only valid rows, ignore the 'Not assigned' Borough values

In [8]:
toronto_df = toronto_df[toronto_df['Borough']!= 'Not assigned']
toronto_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Now, group DataFrame by _Postcode_ and _Borough_ to join the multiple _Neighbourhood_ values into single one

In [9]:
toronto_df_group = toronto_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].agg(', '.join).reset_index(name='Neighbourhood')

#### Finnaly, check again

In [10]:
toronto_df_group.shape

(103, 3)

#### Chenge _Not assigned_ value on M7A Postcode with _Borough's_ value

In [12]:
for index, value in toronto_df_group.iterrows():
    if value['Neighbourhood'] == 'Not assigned':
        new_value = value['Borough']
        value['Neighbourhood'] = new_value

#### Let's see the M7A postcode

In [13]:
toronto_df_group[toronto_df_group['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Everything semms allright

In [14]:
#### Now, we try to request Toronto's location by Fourquesst API

In [15]:
CLIENT_ID = 'ZISPWCOUB0LBELFZ1MZVW5PATP4J4SRMYNJ5WRYEWMIJXD3P' # your Foursquare ID
CLIENT_SECRET = 'BRGEUPNWAKXALC4NN1UEL0AOFZZBQH5OELNDN4FSKNKQFICC' # your Foursquare Secret
VERSION = '20191024'  #20180604
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZISPWCOUB0LBELFZ1MZVW5PATP4J4SRMYNJ5WRYEWMIJXD3P
CLIENT_SECRET:BRGEUPNWAKXALC4NN1UEL0AOFZZBQH5OELNDN4FSKNKQFICC


In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="foursquare_agent", timeout=3)

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


In [29]:
'''for index, row in toronto_df_group.iterrows():
    address = row['Borough']
    geolocator = Nominatim(user_agent="foursquare_agent", timeout=1)
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    toronto_df_group['latidude'] = latitude
    toronto_df_group['longitude'] = longitude '''

'for index, row in toronto_df_group.iterrows():\n    address = row[\'Borough\']\n    geolocator = Nominatim(user_agent="foursquare_agent", timeout=1)\n    location = geolocator.geocode(address)\n    latitude = location.latitude\n    longitude = location.longitude\n    toronto_df_group[\'latidude\'] = latitude\n    toronto_df_group[\'longitude\'] = longitude '

In [30]:
toronto_df_group.head()

,Postcode,Borough,Neighbourhood,latidude,longitude
0,M1B,Scarborough,"Rouge, Malvern",54.282001,-0.401187
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",54.282001,-0.401187
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",54.282001,-0.401187
3,M1G,Scarborough,Woburn,54.282001,-0.401187
4,M1H,Scarborough,Cedarbrae,54.282001,-0.401187


#### It seems not working...:((  /br
remove 'latidude', 'longitude' columns

In [32]:
toronto_df_group = toronto_df_group.drop(columns=['latidude', 'longitude'])
toronto_df_group.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### We have to mannualy download and import into Pandas df the Toronto's coordonates

In [31]:
toronto_coord = pd.read_csv('Geospatial_Coordinates.csv')
toronto_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Let's check the dataframe's shapes

In [34]:
print(toronto_df_group.shape, toronto_coord.shape,
    toronto_df_group.shape==toronto_coord.shape)

(103, 3) (103, 3) True


#### Now, let's copy the Latitude and Longitude columns to our DataFrame

In [36]:
(toronto_df_group['Latitude'], toronto_df_group['Longitude']) = (toronto_coord['Latitude'], toronto_coord['Longitude'])

In [38]:
toronto_df_group.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
